# Classificação de Imagens de 03 classes de plantas por tipos de doenças, com Transfer Learning

#### Objetivo: Realizar o Teste de Inferência com imagens de testes de classe, para o cenário do Caso de Teste 03.

##### Classes Consideradas:
###### Grape___Black_rot
###### Grape___Esca_(Black_Measles)
###### Grape___healthy


## Consultas:
##### https://keras.io/applications/
##### https://www.tensorflow.org/tutorials/load_data/images
##### https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
##### https://keras.io/api/data_loading/
##### https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python

## Leitura do local da base de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
workdir_path = '/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Mounted at /content/drive


In [2]:
os.path.abspath(os.getcwd())

'/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive'

In [3]:
# Base de imagens para treino com total de 4659 imagens (classes de tipos de platas)
trainpath = workdir_path + '/classeuva/Train'

# Base de imagens para validação com total 1169 imagens (classes de tipos de platas)
validpath = workdir_path + '/classeuva/Valid'

# Base de imagens para validação com total de 30 imagens (classes de tipos de platas)
testpath = workdir_path + '/classeuva/Test'


## Leitura de Bibliotecas

In [4]:
from tensorflow.keras.models import load_model
import argparse
import pickle
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import numpy as np
import logging, os
import sys 
from keras.preprocessing import image
import tensorflow as tf
import math
import operator
from pathlib import Path


## Leitura do modelo do respectivo cenário 04-i2 para o Caso de Teste 03

In [5]:
# Ler modelo treinado gravado para submeter aos dados de testes
modelosalvo=tf.keras.models.load_model('/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive/Inferência-TestesModelos/Caso de Teste 03/Resnet50-cenário04-i2/Modelo_Salvo_1/Resnet50.h5')


## Carrega as Imagens de Testes Selecionadas

In [6]:
# Prepara variáveis para imagens do tipo Black_rot

Black_rot_path = testpath +'/Black_rot/' # Caminho das imagens

imagens_Black_rot = [] # Inicializa uma lista para carregar as imagens


In [7]:
# Carrega as imagens para do tipo Black_rot
for img in os.listdir(Black_rot_path):                                 # Cria um loop
        img = os.path.join(Black_rot_path, img)                        
        img = tf.keras.utils.load_img(img, target_size=(224,224))      # coloca a imagem no shape 
        img = tf.keras.utils.img_to_array(img)                         # transforma em um array
        img = np.expand_dims(img, axis=0)                           
        imagem_normalizada = np.array(img, dtype="float") / 255.0      # normaliza a imagem dividindo por 255
        imagens_Black_rot.append(imagem_normalizada)                   # carrega a imagem para a lista
        
imagens_Black_rot = np.vstack(imagens_Black_rot)                         

## Previsão Dados de Teste (classe Black-Hot)

In [8]:
# faz a previsao das imagens de teste do tipo Black_rot
dados_previsao_imagens_Black_rot = []                                 # inicializa uma lista para colocar o reultado da previsao das imagens na lista
nome_arquivos = []                                                  # inicializa uma lista para colocar o nome dos arquivos
for arquivo in os.listdir(Black_rot_path):                            # inicializa um loop (para cada arquivo dentro da pasta)
    pred_result = modelosalvo.predict(imagens_Black_rot)              # faz a previsao de cada arquivo
    dados_previsao_imagens_Black_rot.append(pred_result)              # coloca os dados da previsão na lista
    nome_arquivos.append(arquivo)                                   # coloca o nome do arquivo na lista

1/1 [==============================] - 0s 32ms/step


In [9]:
dados_previsao_imagens_Black_rot[0]

array([[3.5394377e-01, 6.4603084e-01, 2.5329999e-05],
       [3.5884833e-01, 6.4108980e-01, 6.1861312e-05],
       [8.5483980e-01, 1.3616782e-01, 8.9923609e-03],
       [3.5722288e-01, 6.4271462e-01, 6.2409214e-05],
       [1.9946390e-01, 8.0053365e-01, 2.5025897e-06],
       [8.7491953e-01, 1.0082678e-01, 2.4253672e-02],
       [3.5066119e-01, 6.4921337e-01, 1.2544695e-04],
       [7.7039206e-01, 2.2870752e-01, 9.0036524e-04],
       [7.7862692e-01, 2.2062518e-01, 7.4780296e-04],
       [7.5269216e-01, 2.4642347e-01, 8.8438840e-04]], dtype=float32)

In [10]:
# Define os labels R:Grape___Black_rot, M:Grape___Esca_(Black_Measles) e H:Grape___healthy' na mesma ordem de classes do modelo na etapa de load.
# Previsão dados de testes selecionados

label_file_path = os.path.join("RMH", 'labels') 

#SORT
top_k = []
pred = [] 
for i in range(len(dados_previsao_imagens_Black_rot)):
            ordem = dados_previsao_imagens_Black_rot[i][0].argsort()[-len(dados_previsao_imagens_Black_rot[i][0]):][::-1]
            top_k.append(ordem)
            top = top_k[i][:10]
            print(nome_arquivos[i])
            for node_id in top:
                planta_string = label_file_path[node_id]
                score = dados_previsao_imagens_Black_rot[i][0][node_id]
                print('%s (score = %.5f)' % (planta_string, score))

e6444b5c-2944-46c8-a188-b9fbb366a931___FAM_B.Rot 0655_flipLR.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
e95869c1-adf0-40ca-aa7c-14f930d63420___FAM_B.Rot 3537.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
e20443d6-a0a5-4750-a239-bdd3d2f6e69e___FAM_B.Rot 0441_flipLR.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
efe1eba2-52c3-4399-a851-3095e4d1e86a___FAM_B.Rot 0633.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
f0cb2eb6-70cf-4270-9091-0d9ad45eee5c___FAM_B.Rot 3223.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
ed40b1ed-928e-4435-b192-8c7b68e1349e___FAM_B.Rot 3131.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
e441a2b0-1428-43b4-bbfc-a938f50e4e39___FAM_B.Rot 0499.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
f98da630-3128-4702-8e92-60cb024305e5___FAM_B.Rot 5082.JPG
M (score = 0.64603)
R (score = 0.35394)
H (score = 0.00003)
f241eaeb-5b01-4357-840e-3b7e4ff1a539___FAM